
<h1 style="background: linear-gradient(to right, #49A, #0FB); color: white; padding: 20px;">Program 4 Implementation: Artificial Neural Network</h1>

<ol start="4">
    <li>Build an Artificial Neural Network by implementing the Backpropagation 
algorithm and test the same using appropriate data sets.  </li>
</ol>



In [9]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Helper functions
def sigmoid(x): return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x): return x * (1 - x)

# Load and preprocess the Iris dataset
def load_and_preprocess_iris():
    iris = load_iris()
    X, y = iris.data, iris.target.reshape(-1, 1)
    y = OneHotEncoder(sparse_output=False).fit_transform(y)
    X = StandardScaler().fit_transform(X)
    return train_test_split(X, y, test_size=0.3, random_state=64)

# Initialize the neural network
def init_net(input_dim, hidden_dim, output_dim):
    np.random.seed(42)
    return (np.random.rand(input_dim, hidden_dim), 
            np.random.rand(hidden_dim, output_dim), 
            np.zeros((1, hidden_dim)), 
            np.zeros((1, output_dim)))

# Forward pass
def forward_pass(X, w_inh, w_ho, b_h, b_o):
    h_in = np.dot(X, w_inh) + b_h
    h_out = sigmoid(h_in)
    o_in = np.dot(h_out, w_ho) + b_o
    o_out = sigmoid(o_in)
    return h_out, o_out

# Backward pass
def backward_pass(X, y, h_out, o_out, w_ho):
    e = o_out - y
    g_o = e * sigmoid_derivative(o_out)
    e_h = np.dot(g_o, w_ho.T)
    g_h = e_h * sigmoid_derivative(h_out)
    return g_o, g_h

# Update weights
def update_weights(X, h_out, g_o, g_h, w_inh, w_ho, b_h, b_o, lr):
    w_ho -= lr * np.dot(h_out.T, g_o)
    b_o -= lr * np.sum(g_o, axis=0, keepdims=True)
    w_inh -= lr * np.dot(X.T, g_h)
    b_h -= lr * np.sum(g_h, axis=0, keepdims=True)
    return w_inh, w_ho, b_h, b_o

# Train the neural network
def train_ann(X_train, y_train, input_dim, hidden_dim, output_dim, epochs, lr):
    w_inh, w_ho, b_h, b_o = init_net(input_dim, hidden_dim, output_dim)
    for epoch in range(epochs):
        h_out, o_out = forward_pass(X_train, w_inh, w_ho, b_h, b_o)
        g_o, g_h = backward_pass(X_train, y_train, h_out, o_out, w_ho)
        w_inh, w_ho, b_h, b_o = update_weights(X_train, h_out, g_o, g_h, w_inh, w_ho, b_h, b_o, lr)
        if epoch % 100 == 0: print(f"Epoch {epoch}, Loss: {np.mean((y_train - o_out) ** 2):.4f}")
    return w_inh, w_ho, b_h, b_o

# Test the neural network
def test_ann(X_test, y_test, w_inh, w_ho, b_h, b_o):
    _, o_out = forward_pass(X_test, w_inh, w_ho, b_h, b_o)
    preds = np.argmax(o_out, axis=1)
    true_labels = np.argmax(y_test, axis=1)
    acc = np.mean(preds == true_labels) * 100
    print(f"Test Accuracy: {acc:.2f}%")

# Main function
def main():
    X_train, X_test, y_train, y_test = load_and_preprocess_iris()
    input_dim, hidden_dim, output_dim = X_train.shape[1], 5, y_train.shape[1]
    epochs, lr = 500, 0.01
    w_inh, w_ho, b_h, b_o = train_ann(X_train, y_train, input_dim, hidden_dim, output_dim, epochs, lr)
    test_ann(X_test, y_test, w_inh, w_ho, b_h, b_o)

if __name__ == "__main__":
    main()


Epoch 0, Loss: 0.3929
Epoch 100, Loss: 0.1074
Epoch 200, Loss: 0.0881
Epoch 300, Loss: 0.0683
Epoch 400, Loss: 0.0478
Test Accuracy: 95.56%
